In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
sys.path.append(os.path.join(os.getcwd(), ".."))
from metrics.methods.tsd_metrics import TSD_calculator_time_series
path_formatted_glasgow = "/workspaces/ecg_evaluation/data/20221006_physio_quality/ECG_recording"
path_csv_ref_label = "/workspaces/ecg_evaluation/data/20221006_physio_quality/Notes.txt"

In [ ]:
###Read one file (associated with one pathology):

def The_ECG_getter(path_signal,path_label):

    df_label = pd.read_csv(path_label,sep = "\t",header = None,names = ["Characteristic"])
    labels = df_label.to_numpy()
    label = np.array([labels[i,0].replace(f"{i+1}.","") for i in range(labels.shape[0])])
    dico_ECG  = {}
    name_lead = ['I','II','III','aVR','aVF','aVL','V1','V2','V3','V4','V5','V6']
    for i in range(len(label)):
        signal = pd.read_csv(path_signal + f"/ecg_{i+1}.txt",sep = ",",names = name_lead)
        signal_np = signal.to_numpy()
        dico_ECG[label[i]] = signal_np
    return label,dico_ECG,name_lead

In [ ]:
lab,dic_signal,name_lead = The_ECG_getter(path_formatted_glasgow,path_csv_ref_label)


In [ ]:
print(lab)
print(len(lab))

In [ ]:
def Plot_TSD_signal(signal,lead,interval,patho_name):
    t = np.linspace(0,len(signal)/500,len(signal))
    Ds,Dm = TSD_calculator_time_series(signal,100,500)
    plt.figure()
    _,ax = plt.subplots(nrows = 2, ncols=2,figsize=(20,15))
    ax[0,0].plot(t,signal.copy(),label = "TSD score = {0:.2f}".format(Dm))
    ax[0,0].set_title(f"Time Evolution of Lead {lead} with {patho_name}")
    ax[0,0].set_xlabel("Time (sec)")
    ax[0,0].set_ylabel("Volatge Amplitude")
    ax[0,0].legend()
    ax[0,0].grid()
    ax[1,0].plot(np.linspace(0,int(len(Ds)/500),len(Ds)),Ds,label = "TSD score = {0:.2f}".format(Dm))
    ax[1,0].plot(t,np.ones_like(t)*Dm,"--k",label = "Mean TSD value")
    ax[1,0].set_title(f"TSD evolution for {lead}  using a segment length of {100}")
    ax[1,0].grid()
    ax[1,0].legend()
    ax[1,0].set_xlabel("Lags")
    ax[1,0].set_ylabel("TSD value")
    ax[0,1].plot(t,signal.copy(),label = "TSD score = {0:.2f}".format(Dm))
    ax[0,1].set_title(f"Time Evolution of Lead {lead} for an time interval of {interval} with {patho_name}")
    ax[0,1].set_xlabel("Time (sec)")
    ax[0,1].set_ylabel("Volatge Amplitude")
    ax[0,1].set_xlim(interval)
    ax[0,1].legend()
    ax[0,1].grid()
    ax[1,1].plot(np.linspace(0,int(len(Ds)/500),len(Ds)),Ds,label = "TSD score = {0:.2f}".format(Dm))
    ax[1,1].set_title(f"TSD evolution for {lead}  using a segment length of {100} for a time interval of {interval}")
    ax[1,1].grid()
    ax[1,1].plot(t,np.ones_like(t)*Dm,"--k",label = "Mean TSD value")
    ax[1,1].legend()
    ax[1,1].set_xlim(interval)
    ax[1,1].set_xlabel("Lags")
    ax[1,1].set_ylabel("TSD value")
    plt.show()

In [ ]:
signal = dic_signal[lab[7]]
for i in range(len(name_lead)):
    Plot_TSD_signal(signal[:,i],name_lead[i],[2,6],lab[7])